In [1]:
import re

def extract_case_answer(response):
    response = response.replace('*', '')
    match = re.search(r'The correct answer is.*?(CASE_\d+)', response, re.IGNORECASE)
    return match.group(1) if match else None

In [3]:
def test_extract_case_answer():
    test_cases = [
        ("The correct answer is CASE_123.", "CASE_123"),
        ("The correct answer is: (CASE_456)", "CASE_456"),
        ("The correct answer is <CASE_789>", "CASE_789"),
        ("The correct answer is... maybe... CASE_000?", "CASE_000"),
        ("Something unrelated. The correct answer is 🌟(CASE_321)", "CASE_321"),
        # ("The correct answer is\n\nCASE_222\nbecause...", "CASE_222"),
        ("the correct answer is~~~<<CASE_999>>", "CASE_999"),
        ("No answer provided.", None),
        ("Answer: CASE_555", None),
        ("*The correct answer is* (CASE_666)", "CASE_666"),
    ]

    for i, (input_text, expected_output) in enumerate(test_cases):
        result = extract_case_answer(input_text)
        assert result == expected_output, f"Test case {i+1} failed: expected {expected_output}, got {result}"
        print(f"Test case {i+1} passed.")

test_extract_case_answer()


Test case 1 passed.
Test case 2 passed.
Test case 3 passed.
Test case 4 passed.
Test case 5 passed.
Test case 6 passed.
Test case 7 passed.
Test case 8 passed.
Test case 9 passed.


In [5]:
from datasets import load_dataset
import random

# 加载本地缓存数据集
dataset = load_dataset(
    "/home/ubuntu/Web/hzy/LooGLE-v2/datasets/LooGLE-v2/GraphPKU___loo_gle-v2",
    split="test"
)

# 转为 list 后再随机采样
sampled = random.sample(list(dataset), 20)
instructions = [item["instruction"] for item in sampled]

# 打印结果
for i, ins in enumerate(instructions, 1):
    print(f"{i}. {ins}")


1. You are a senior financial analyst. You will be provided with certain fiscal year's annual reports for one or more companies. In the provided financial reports, any data enclosed in parentheses () within tables is to be interpreted as a negative value. The text below contains all the 10-K annual reports in the format <FILE: filename> followed by the file content. <text> {context} </text> Your task is to analyze the financial information and answer the following question: <question> {question} </question>.
2. Now, I'm going to present you a textual description of a Counter-Strike-2 (CS2) game and a question. Please read the text carefully and answer the question. CS2 is a tactical first-person shooter game featuring two teams, each consisting of five players. In the first half of the match, one team plays the role of the Terrorists ([T]), and the other plays the role of the Counter-Terrorists ([CT]). After the first 13 rounds, the two teams switch sides. The goal of the Terrorists is

In [2]:
import json

input_path = "/home/ubuntu/Web/hzy/LooGLE-v2/results_cot/GLM-4-9b-chat.jsonl"
output_path = input_path  # 若想保留原始文件，可以改为别的路径

with open(input_path, 'r', encoding='utf-8') as fin, open(output_path + ".tmp", 'w', encoding='utf-8') as fout:
    for line in fin:
        item = json.loads(line)
        if item.get("pred_answer") is not None:
            fout.write(json.dumps(item, ensure_ascii=False) + "\n")

# 替换原文件
import os
os.replace(output_path + ".tmp", output_path)


In [ ]:
import gymnasium as gym

# Initialise the environment
env = gym.make("LunarLander-v3", render_mode="human")

# Reset the environment to generate the first observation
observation, info = env.reset(seed=42)
for _ in range(1000):
    # this is where you would insert your policy
    action = env.action_space.sample()

    # step (transition) through the environment with the action
    # receiving the next observation, reward and if the episode has terminated or truncated
    observation, reward, terminated, truncated, info = env.step(action)

    # If the episode has ended then we can reset to start a new episode
    if terminated or truncated:
        observation, info = env.reset()

env.close()